In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random



curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

68 documents
39 classes ['', 'Are support programmes and counselling offered for free at DSSA? ', 'Criminal Case Management System (CCMS)', 'How can I get help on child maintenance issues? ', 'How do I get help when I am faced with family violence? ', 'How to institute a Private Prosecutor', 'I am interested in joining the Attorney-Generalâ€™s Chambers.  Where and how do I apply for a job? ', 'I have a dispute with my neighbor as there is loud music playing from his house all night.  Can I ask the Attorney-General to take action against him on my behalf?', 'I run a construction company and have participated in a tender exercise to secure a contract with the Government.  The tender exercise has closed.  Can I now negotiate with the Government on the contractual terms and the price?', 'If I have a legal problem, can AGC advise me?', 'Must all representations on behalf of accused persons be submitted to AGC through a hard copy letter?', 'Overseas offence', 'Personal Injury', 'Private Pros

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [11]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1101, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 9850  | total loss: 0.13982 | time: 0.012s
| Adam | epoch: 1095 | loss: 0.13982 - acc: 0.9901 -- iter: 32/68


In [12]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
['', 'Are support programmes and counselling offered for free at DSSA? ', 'Criminal Case Management System (CCMS)', 'How can I get help on child maintenance issues? ', 'How do I get help when I am faced with family violence? ', 'How to institute a Private Prosecutor', 'I am interested in joining the Attorney-Generalâ€™s Chambers.  Where and how do I apply for a job? ', 'I have a dispute wi

In [13]:
print(model.predict([p]))


[[4.0830921e-02 1.2790265e-05 1.7791596e-07 4.1618546e-06 7.0823221e-06
  2.2038825e-04 3.9848641e-07 1.3713448e-07 1.9438225e-07 7.0238668e-05
  5.7561090e-03 9.4372714e-05 1.6827431e-02 3.6592305e-06 4.8837901e-06
  9.4757081e-05 1.3143020e-05 5.6099348e-06 5.5002799e-04 3.9929334e-02
  6.2666484e-04 2.2073302e-03 1.0157297e-02 1.0784854e-03 3.1065443e-04
  1.1461031e-05 9.0962844e-03 8.4640305e-06 2.4754352e-06 1.4258397e-07
  1.5093558e-07 9.3412682e-06 1.4997281e-04 1.0370709e-03 1.6282471e-01
  3.2826111e-01 3.9085022e-05 5.1192689e-04 3.7924150e-01]]


In [14]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )